# به نام خدا

# فاز 1

In [58]:
import cv2
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [59]:


def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    plates = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name.lower() == 'vehicle plate':
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            plates.append((xmin, ymin, xmax, ymax))
    
    return plates

def extract_plates(image_path, annotation_path, output_path):
    # خواندن تصویر
    image = cv2.imread(image_path)
    if image is None:
        print(f"خطا در خواندن تصویر: {image_path}")
        return
    
    # خواندن آنوتیشن‌ها
    plates = parse_annotation(annotation_path)
    if not plates:
        print(f"هیچ پلاکی در فایل آنوتیشن یافت نشد: {annotation_path}")
        return
    
    # استخراج و ذخیره هر پلاک
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    for i, (xmin, ymin, xmax, ymax) in enumerate(plates):
        plate_img = image[ymin:ymax, xmin:xmax]
        
        # ذخیره پلاک استخراج شده
        output_file = os.path.join(output_path, f"{base_name}.png")
        cv2.imwrite(output_file, plate_img)

def process_dataset(base_dir, output_dir):
    # مسیرهای ورودی
    images_dir = os.path.join(base_dir, "Vehicle Plates", "Vehicle Plates")
    annotations_dir = os.path.join(base_dir, "Vehicle Plates annotations", "Vehicle Plates annotations")
    
    # بررسی وجود پوشه‌ها
    if not os.path.exists(images_dir):
        print(f"پوشه تصاویر یافت نشد: {images_dir}")
        return
    if not os.path.exists(annotations_dir):
        print(f"پوشه آنوتیشن‌ها یافت نشد: {annotations_dir}")
        return
    
    # لیست تمام فایل‌های تصویر
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # ایجاد پوشه خروجی اگر وجود نداشته باشد
    os.makedirs(output_dir, exist_ok=True)
    
    # پردازش هر تصویر با نوار پیشرفت
    for img_file in tqdm(image_files, desc="پردازش تصاویر"):
        base_name = os.path.splitext(img_file)[0]
        image_path = os.path.join(images_dir, img_file)
        annotation_path = os.path.join(annotations_dir, f"{base_name}.xml")
        
        if os.path.exists(annotation_path):
            extract_plates(image_path, annotation_path, output_dir)
        else:
            print(f"فایل آنوتیشن یافت نشد: {annotation_path}")

if __name__ == "__main__":
    # مسیرهای ورودی و خروجی
    base_dir = "Plates2"
    output_dir = "output_phase1"
    
    # اجرای پردازش
    process_dataset(base_dir, output_dir)
    
    print(f"پردازش فاز اول با موفقیت به پایان رسید. پلاک‌های استخراج شده در پوشه '{output_dir}' ذخیره شدند.")

پردازش تصاویر: 100%|██████████| 217/217 [00:00<00:00, 301.14it/s]

پردازش فاز اول با موفقیت به پایان رسید. پلاک‌های استخراج شده در پوشه 'output_phase1' ذخیره شدند.


# فاز 2

### درست کردن کجی و سیاه سفید کردن

In [18]:
import os
import cv2
import numpy as np

input_folder = 'output_phase1'
output_folder = 'output_phase2'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def angle_from_min_area_rect(thresh):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    largest = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(largest)
    angle = rect[-1]
    if angle < -45:
        angle += 90
    elif angle > 45:
        angle -= 90
    return angle

def angle_from_hough(gray):
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=30, maxLineGap=10)
    if lines is None:
        return None
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        if -45 < angle < 45:
            angles.append(angle)
    if not angles:
        return None
    return np.median(angles)

def correct_skew_combined(gray_image, angle_threshold=3):
    # Preprocess
    _, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    thresh = 255 - thresh

    # Step 1: try minAreaRect
    angle = angle_from_min_area_rect(thresh)

    # Step 2: fallback to HoughLines if angle too small or None
    if angle is None or abs(angle) < angle_threshold:
        angle = angle_from_hough(gray_image)

    # اگر همچنان زاویه معنادار نبود، تصویر را برگردان
    if angle is None or abs(angle) < angle_threshold:
        return gray_image

    # Rotate
    (h, w) = gray_image.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(gray_image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# پردازش همه تصاویر
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        path = os.path.join(input_folder, filename)
        image = cv2.imread(path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        corrected = correct_skew_combined(gray)

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, corrected)

print("✅ اصلاح زاویه با ترکیب minAreaRect + Hough انجام شد.")


✅ اصلاح زاویه با ترکیب minAreaRect + Hough انجام شد.


### کات دادن یک هشتم اول پلاک

In [7]:
import os
from PIL import Image

# مسیرهای ورودی و خروجی
input_dir = 'output_phase2'
output_dir = 'output_phase2.1'

# ساخت پوشه خروجی اگر وجود نداشت
os.makedirs(output_dir, exist_ok=True)

# پردازش هر تصویر در پوشه ورودی
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)

        # ابعاد تصویر
        width, height = img.size

        # عرض هر تکه
        slice_width = width // 8

        # برش تصویر: حذف تیکه اول (از x=0 تا x=slice_width)
        # نگه داشتن بخش باقی‌مانده (از x=slice_width تا x=width)
        cropped_img = img.crop((slice_width, 0, width, height))

        # ذخیره تصویر خروجی
        output_path = os.path.join(output_dir, filename)
        cropped_img.save(output_path)

print("پردازش تمام شد.")


پردازش تمام شد.


### باینری کردن عکس ها

In [60]:
import os
import cv2
import numpy as np

input_folder = 'output_phase2.1'
output_folder = 'output_phase2.2'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # 1. نویززدایی با فیلتر غیرمحلی (Non-local Means)
        denoised = cv2.fastNlMeansDenoising(img, h=10, templateWindowSize=7, searchWindowSize=21)
        
        # 2. افزایش کنتراست با CLAHE (بهتر از اکولایزیشن ساده)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        contrast_enhanced = clahe.apply(denoised)
        
        # 3. فیلتر گوسی برای نرم کردن (با هسته کوچکتر)
        blurred = cv2.GaussianBlur(contrast_enhanced, (3,3), 0)
        
        # 4. آستانه‌گذاری تطبیقی با پارامترهای بهینه‌تر
        binary_img = cv2.adaptiveThreshold(
            blurred,
            255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            11,  # اندازه بلوک - باید کوچکتر باشد برای حفظ جزئیات
            2    # مقدار ثابت - کاهش یافته برای جلوگیری از سیاه شدن نواحی روشن
        )
        
        # 5. مورفولوژی برای حذف نویزهای کوچک
        kernel = np.ones((2,2), np.uint8)
        processed = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernel)
        
        # معکوس کردن تصویر برای داشتن متن سفید روی پس زمینه سیاه
        final_img = cv2.bitwise_not(processed)
        
        # ذخیره خروجی
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, final_img)

print("پردازش پیشرفته و تبدیل به باینری با حفظ جزئیات انجام شد.")

پردازش پیشرفته و تبدیل به باینری با حفظ جزئیات انجام شد.


In [ ]:
import os
import cv2
import numpy as np

input_folder = 'output_phase2.1'
output_folder = 'output_phase2.2'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # 1. پیش‌پردازش اولیه با حفظ جزئیات
        denoised = cv2.fastNlMeansDenoising(img, h=5, templateWindowSize=7, searchWindowSize=21)
        
        # 2. افزایش کنتراست هوشمند
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(12,12))
        contrast_enhanced = clahe.apply(denoised)
        
        # 3. بهبود لبه‌ها با فیلتر ویژه
        blurred = cv2.bilateralFilter(contrast_enhanced, d=9, sigmaColor=75, sigmaSpace=75)
        
        # 4. روش ترکیبی آستانه‌گذاری
        # مرحله اول: Adaptive Threshold
        binary_adaptive = cv2.adaptiveThreshold(
            blurred,
            255,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY_INV,
            blockSize=9,
            C=2
        )
        
        # مرحله دوم: Otsu Threshold
        _, binary_otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # مرحله سوم: ترکیب با وزن‌دهی
        combined = cv2.addWeighted(binary_adaptive, 0.7, binary_otsu, 0.3, 0)
        
        # 5. بهبود نهایی با تکنیک Super-Resolution (اختیاری)
        # اگر opencv با contrib نصب باشد:
        # super_res = cv2.dnn_superres.DnnSuperResImpl_create()
        # super_res.readModel('EDSR_x4.pb')
        # super_res.setModel('edsr', 4)
        # final_img = super_res.upsample(combined)
        
        # 6. ذخیره نتیجه
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cv2.bitwise_not(combined))

print("پردازش حرفه‌ای با حفظ حداکثر کیفیت انجام شد.")

پردازش حرفه‌ای با حفظ حداکثر کیفیت انجام شد.


### در اوردن کاراکتر های هر عکس

In [50]:
import cv2
import os

input_folder = 'output_phase2.1'
output_folder = 'output_phase2.2'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # استفاده از آستانه گذاری Otsu برای باینری شدن دقیق
        _, img_bin = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # ذخیره موقت تصویر باینری برای اطمینان از کیفیت
        cv2.imwrite(os.path.join(output_folder, f'{os.path.splitext(filename)[0]}_binary.png'), img_bin)

        # اجرای connectedComponents روی تصویر باینری
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_bin, connectivity=8)

        components = []
        for i in range(1, num_labels):
            x, y, w, h, area = stats[i]
            if area > 100 and w > 5 and h > 20:
                components.append((x, y, w, h))

        components = sorted(components, key=lambda b: b[0])

        if len(components) != 8:
            print(f"Warning: {filename} has {len(components)} components detected, expected 8")

        for idx, (x, y, w, h) in enumerate(components):
            char_img = img_bin[y:y+h, x:x+w]
            out_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}_char_{idx+1}.png')
            cv2.imwrite(out_path, char_img)

print("Finished processing images.")


Finished processing images.
